In [1]:
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
!pip install gym[box2d]

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  freeglut3 libbsd0 libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2
  libdrm-radeon1 libdrm2 libedit2 libelf1 libgl1 libgl1-mesa-dri
  libgl1-mesa-glx libglapi-mesa libglu1-mesa libglvnd0 libglx-mesa0 libglx0
  libllvm10 libpciaccess0 libpython-stdlib libpython2.7-minimal
  libpython2.7-stdlib libsensors4 libx11-6 libx11-data libx11-xcb1 libxau6
  lib

EasyProcessError: start error <EasyProcess cmd_param=['Xvfb', '-help'] cmd=['Xvfb', '-help'] oserror=[Errno 2] No such file or directory: 'Xvfb': 'Xvfb' return_code=None stdout="None" stderr="None" timeout_happened=False>

In [2]:
import os
import time
import pandas as pd
from agent import *
import pickle

### Fine tune discrete agents
For sake of time the hyperparameters here will be used for the other models as well

In [3]:
%%time
SAVE_PATH = '/tf/kfir/nir/sokoban_rl/hyperparam_tunings.pkl'
EPOCHS = 1500
TRAJ_LEN = 1000
BETA = 1e-3
PRINT_INTERVAL = 100
LOG_INTERVAL = 0
SCHED_GAMMA = 0.9
SCHED_INTERVAL = 300
CONE_TRICK = True

discrete_models = ['SplitDiscreteActorCritic']
num_discretes = [6, 8, 10, 12]
discounts = [0.99, 0.97]
traj_lengths = [1000, 250]
lrs = [0.22, 0.0018, 0.0014, 0.001, 0.0006]
hidden_sizes = [512, 256, 128]
discrete_agents = {}

for model_name in discrete_models:
    for num_discrete in num_discretes:
        for discount in discounts:
            for traj in traj_lengths:
                for lr in lrs:
                    for hidden_size in hidden_sizes:
                        agent_name = '{}_{}_{}_{}_{}_{}'.format(model_name, num_discrete, discount, traj, lr, hidden_size)
                        print('#'*20, agent_name, '#'*20)    
                        start = time.time()
                        envs = [utils.EnvWrapper('LunarLanderContinuous-v2', utils.ObsType['BOX2D'], 
                                                utils.ActionType['DISCRETIZIED'], 
                                                5000, num_discrete=num_discrete,
                                                cone_trick=CONE_TRICK) for _ in range(2)]
                        env_gen = utils.AsyncEnvGen(envs, 1)
                        model = getattr(models, model_name)(envs[0].obs_size, envs[0].num_actions, 
                                                            hidden_size=hidden_size, num_discrete=num_discrete)
                        save_path = './model.pkl'
                        log_path = './log.log'
                        agent = A2CAgent(model, save_path, log_path)

                        try:
                            agent.train(epochs=EPOCHS,
                                        trajectory_len=traj,
                                        env_gen=env_gen,
                                        lr=lr,
                                        discount_gamma=discount,
                                        scheduler_gamma=SCHED_GAMMA,
                                        beta=BETA,
                                        print_interval=PRINT_INTERVAL,
                                        log_interval=LOG_INTERVAL,
                                        scheduler_interval=SCHED_INTERVAL)
                        except Exception as e:
                            raise e
                        finally:
                            end = time.time()
                            utils.kill_process(env_gen)
                            discrete_agents[agent_name] = agent
                        print('----- training took {:.3f} minutes -----\n'.format((end - start)/60))

with open(SAVE_PATH, 'wb') as f:
    pickle.dump(discrete_agents, f)                    

#################### SplitDiscreteActorCritic_6_0.99_1000_0.22_512 ####################
Using CUDA
episode: 100, stats for last 100 episodes:	avg reward: -433.927	avg length: 64.290	 avg time: 0.105
episode: 200, stats for last 100 episodes:	avg reward: -432.850	avg length: 63.210	 avg time: 0.104
episode: 300, stats for last 100 episodes:	avg reward: -436.855	avg length: 65.120	 avg time: 0.107
stepped scheduler, new lr: 0.19800
episode: 400, stats for last 100 episodes:	avg reward: -445.498	avg length: 65.280	 avg time: 0.108
episode: 500, stats for last 100 episodes:	avg reward: -438.511	avg length: 65.240	 avg time: 0.107
episode: 600, stats for last 100 episodes:	avg reward: -425.361	avg length: 62.200	 avg time: 0.102
stepped scheduler, new lr: 0.17820
episode: 700, stats for last 100 episodes:	avg reward: -426.800	avg length: 63.810	 avg time: 0.104
episode: 800, stats for last 100 episodes:	avg reward: -449.900	avg length: 64.600	 avg time: 0.106
episode: 900, stats for last 10

### Visualization

In [19]:
len(discrete_agents)
a = discrete_agents['SplitDiscreteActorCritic_6_0.99_1000_0.0018_256']
means = []
for agent_name in discrete_agents:
    agent = discrete_agents[agent_name]
    means.append((agent_name, np.mean(agent.all_rewards[500:1100])))
sorted_means = sorted(means, key=lambda tup: tup[-1])[::-1]

In [20]:
sorted_means

[('SplitDiscreteActorCritic_12_0.97_1000_0.0018_512', 69.92015922673431),
 ('SplitDiscreteActorCritic_8_0.97_1000_0.0014_512', 52.23887745118383),
 ('SplitDiscreteActorCritic_10_0.97_250_0.0018_512', 25.346733632064186),
 ('SplitDiscreteActorCritic_8_0.99_1000_0.0018_512', 12.310947706782414),
 ('SplitDiscreteActorCritic_6_0.97_250_0.001_512', 11.743650461927519),
 ('SplitDiscreteActorCritic_10_0.99_1000_0.001_512', -13.666838961136273),
 ('SplitDiscreteActorCritic_12_0.97_1000_0.0014_256', -14.407398399187253),
 ('SplitDiscreteActorCritic_6_0.97_1000_0.001_512', -25.95494292303923),
 ('SplitDiscreteActorCritic_12_0.97_250_0.0018_512', -32.201290126811216),
 ('SplitDiscreteActorCritic_8_0.97_1000_0.0018_256', -32.36395076335634),
 ('SplitDiscreteActorCritic_6_0.99_250_0.0018_512', -32.71136579355764),
 ('SplitDiscreteActorCritic_8_0.99_250_0.0014_512', -34.1795561887192),
 ('SplitDiscreteActorCritic_12_0.99_1000_0.0018_256', -36.983812352881735),
 ('SplitDiscreteActorCritic_10_0.97_100

In [ ]:
# TODO:
# TURN OFF CONE-TRICK AT EPOCH 700
# try higher penalty in cone trick (-400)
# test gaussians and convs
# use clip_grad for gaussians!
# Finally train the Sokoban

In [ ]:
with open('/content/drive/MyDrive/RL_project/saved_agents/test_agent.pkl', 'rb') as f:
    discrete_agents = pickle.load(f)

In [ ]:
def moving_average(numbers, window_size):
    i = 0
    moving_averages = []
    while i < len(numbers) - window_size + 1:
        this_window = numbers[i : i + window_size]
        window_average = sum(this_window) / window_size
        moving_averages.append(window_average)
        i += 1
    return moving_averages

In [ ]:
keys = [key for key in discrete_agents.keys()]
COLORS = ['b', 'g', 'c', 'm', 'y', 'k']

In [ ]:
fig, axes = plt.subplots(3, 1,  figsize=(10,25))
for num, key in enumerate(discrete_agents):
    axes[num//4].plot(moving_average(discrete_agents[key].all_rewards, 100), color=COLORS[num%4])
[axes[i].legend(keys[q:q+4]) for i, q in enumerate(range(0,len(keys), 4))]

In [ ]:
# env = gym.make('CartPole-v0')
env.env.max

In [ ]:
EPOCHS = 100
TRAJ_LEN = 300
SCHED_GAMMA = 0.98
BETA = 1e-3
PRINT_INTERVAL = 500
LOG_INTERVAL = 0
SCHED_INTERVAL = 1000

gaussian_models = ['GaussianActorCritic', 'GaussianConvActorCritic']
lrs = [1e-3, 3e-4, 1e-5]
num_discretes = [100, 500, 10]
discounts = [0.8,0.9,0.99]
traj_lengths = [5,10,50,200,500]

discrete_agents = {}

for model_name in discrete_models:
    for num_discrete in num_discretes:
        for discount in discounts:
            for traj in traj_lengths:
                for lr in lrs:
                    agent_name = '{}_{}_{}_{}_{}'.format(model_name, num_discrete, discount, traj, lr)
                    print('#'*20, agent_name, '#'*20)    
                    start = time.time()
                    envs = [utils.EnvWrapper('LunarLanderContinuous-v2', utils.ObsType['BOX2D'], 
                                            utils.ActionType['DISCRETIZIED'], 
                                            5000, num_discrete=num_discrete) for _ in range(2)]
                    env_gen = utils.AsyncEnvGen(envs, 1)
                    model = getattr(models, model_name)(envs[0].obs_size, envs[0].num_actions, num_discrete=num_discrete)
                    save_path = './model.pkl'
                    log_path = './log.log'
                    agent = A2CAgent(model, save_path, log_path)

                    try:
                        agent.train(EPOCHS, traj, env_gen, lr,
                                            discount, SCHED_GAMMA, BETA,
                                            PRINT_INTERVAL, LOG_INTERVAL, SCHED_INTERVAL)
                    finally:
                        end = time.time()
                        utils.kill_process(env_gen)
                        gaussian_models[agent_name] = agent
                    print('----- training took {:.3f} minutes -----\n'.format((end - start)/60))